In [1]:
%load_ext autoreload
%autoreload 2
import os 
os.chdir("..")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from numerics.integration.steps import *
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numerics.utilities.misc import *
from lmfit.models import LorentzianModel

In [3]:
params = give_def_params()
params, exp_path = check_params(params)
[eta, gamma, kappa, omega, n] = params

train_id = 0
periods = 10
ppp = 1000
itraj = 1
rppp_reference = 1
method="rossler"
dt = (2*np.pi/(omega*ppp))

loop over all trajs..

In [4]:
def fit_lorentzian(signals):
    Period = 2*np.pi/omega
    spectra_signal = np.abs(np.fft.fft(signals[:,0]))**2
    freqs_signal = np.fft.fftfreq(n = len(spectra_signal), d= Period/ppp)*(2*np.pi)

    cutoff = 10*omega 
    cond  = np.logical_and(freqs_signal < cutoff, freqs_signal>=0)
    spectra_signal = spectra_signal[cond]
    freqs_signal = freqs_signal[cond]

    mod = LorentzianModel()

    pars = mod.guess(spectra_signal, x=freqs_signal)
    out = mod.fit(spectra_signal, pars, x=freqs_signal)
    
    return out.values["center"]


In [5]:
params = give_def_params()
params, exp_path = check_params(params)
[eta, gamma, kappa, omega, n] = params

train_id = 0
periods = 10
ppp = 1000
itraj = 1
rppp_reference = 1
method="rossler"


fits = []
for itraj in tqdm(range(1,10000)):
    try:
        
        states, covs, signals, params, times, u_th, covs_th = load_data(ppp=ppp, periods=periods, 
                                                         method=method, itraj=itraj, exp_path=exp_path , rppp = rppp_reference)
        fits.append(fit_lorentzian(signals))    
    except Exception:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9999/9999 [27:17<00:00,  6.11it/s]


In [6]:
os.makedirs("fisher_lorentzian/fits_statistics")
with open("fisher_lorentzian/fits_statistics/results_trajs_{}.txt".format(len(fits)),"w") as f:
    f.write(str((np.mean(fits),np.std(fits))))
    f.close()